Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script only solves the model. For a version with unit testing and graphs, refer to "BargainingJulia"

In [1]:
using LinearAlgebra, Statistics

# using Distributions, Expectations, NLsolve, Roots, Random, Plots, Parameters

using DataFrames


In [ ]:
function solve_tre(data, game_parameters, grid=100, hlow=0.01, hhigh=1.0, grid_payment=1000)
    
    #data ####
    #all the data is escalated so as to D = 1.0
    Vmax = data[1]; #firm's maximum reorganization value
    L = data[2]; #firm's maximum liquidation value
    Dj = data[3]; #junior's debt face value
    Ds = 1.0 - Dj; #senior's debt face value

    D = Ds + Dj; #total debt, used to escale costs

    #parameters####
    #parameters include only the parameters necessary to calculate the model, so it does not include θs0, θj0, μ
    ρ = parameters[1]; # (1 - ρ) is the firm's depreciation rate each period
    β = parameters[2]; #inverse of the speed of learning
    c0 = parameters[3]; #fixed cost of going to court
    λj = parameters[4]; #chance of j being called to propose at each period

    #c1 is calibrated. We will start with the same c1 as the authors
    c1 = Float64(0.015); #variable cost for each period at court
    
    
    #Initial definitions: costs, maximum value of the firm, maximum number of periods####


    
    

    #number of periods
    function max_turns(Vmax, L, ρ)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet.
        while(ρ^(t-1) * Vmax > L)
            t = t+1
        end
        return t
    end


    T = max_turns(Vmax, L, ρ)

    ### Important: we will create a game with T+1 periods

    # So we'll be able to include period t==0 in the arrays as index 1

    # So the notation is: period t is represented by the index t-1


    #cost function
    function Ct(t)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end



    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end

    #index 1 corresponds to t=0 and 2 to t=1...
    
    
    

    #liquidation payoffs

    function s_L(t)
        return min(L - Ct(t), Ds)
    end


    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end



    # Skill levels' grid and Probability Mass Function(PMF)

    # Important: we won't use the skill levels directly in the functions, we will use their indexes. For example, index = 40 corresponds to $\theta_{K,t} = 0.4$.

    #grid size
    grid=grid
    hlow = hlow
    hhigh = hhigh
    hvals = LinRange(hlow, hhigh, grid)



    #array to tell the size of the pie at period t, according to reorganization skill levels
    #arguments: (period, θkt)
    U = Array{Float64}(undef, T+1, grid)


    for t in 1:T+1
        for h in 1:grid
            U[t,h] = hvals[h] * Vt(Vmax, ρ, t) - Ct(t)
        end
    end

    #cumulative distribution function
    function cdf(x, lt)

        #return the cdf of x given the lower bound lt
        #geq than 1 because of our discretization method 
        if(x >= 1.0)

            return 1.0

        else

            if(x >= lt)
                return 1.0 - ((1.0-x)^β)/((1-lt)^β)
            else
                return 0.0
            end
        end
    end


    #pmf: each point of the grid is at the center of the cdf
    function pmf_cdf(lt, δ = hlow/2)
        pmf = zeros(grid)

        for (i, h) in enumerate(hvals)
            pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
        end

        return pmf
    end



    #generating the pmfs

    #probability mass function
    pmf = Array{Float64}(undef,grid,grid)

    δ = hlow/2
    for (i, htoday) in enumerate(hvals)
        for (j, htomorrow) in enumerate(hvals)
            pmf[t,:] = cdf(htomorrow+δ, htoday) - cdf(htomorrow-δ, htoday)
        end
    end
    


    #we need a pmf for when we have lkt and want to know θk,t+1
    #we will call it 'pmf2'
    pmf2 = similar(pmf)

    for i in eachindex(hvals)
        for j in eachindex(hvals)
            pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
        end
    end



    # Arrays with the continuation values####

    #continuation values
    #period t (not t+1), θkt, ℓkt, ℓmt
    s_W = zeros(T+1, grid, grid, grid)
    j_W = similar(s_W)


    #optimal payments
    #period, θkt, ℓmt, outputs
    Pst_array = zeros(T, grid, grid, 6) #we won't need payment proposal for period T, a.k.a. "T+1"
    Pjt_array = similar(Pst_array)

    #populating the last period with the liquidation values
    s_W[T+1, :, :, :] .= s_L(T+1); #we need to use .= because it is a broadcast
    j_W[T+1, :, :, :] .= j_L(T+1);

    #cutoff function: upgrade

    function getcutoff_m(t, pkt, hkt, m_W)

        #we extract the diagonal values, where θmt == lmt, to search for the cutoff
        diag_vals = @views diag(m_W[t+1, :, :, hkt])


        #we don't need to worry about negative payment offers because in this case the cutoff will be 1
        #if the pkt is greater than all the continuation values of the adversary, then it will return '101' (grid+1)
        return searchsortedfirst(diag_vals, pkt)
    end


    #function to "filter" the cutoff

    function cutoff_m(t, pkt, hkt, lmt, m_W)
        #for now, it will just tell us if the payment offer is negative

        #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
        if(pkt<0)
             return grid+2
        end

        cmt = getcutoff_m(t, pkt, hkt, m_W)

        return cmt

    end



    #function to select the pmf function as it is writen in the paper

    function h_m(t, lmt, pmf, pmf2)
        #takes the indexes of cutoff and lmt

        #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
        if(t <= 1)
            return pmf[lmt, 1:end]
        else
            return pmf2[lmt, 1:end]
        end
    end



    #version passing global variables as parameters


    grid_payment = grid_payment
    share = LinRange(hlow, hhigh-hlow, grid_payment)

    #testing cutoff in this newer version of Pkt

    #function to calculate the optimal payment offer and directly populate the array

    function Pkt(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U)



        Pkt_grid = Vector{Float64}(undef, length(share)+1)

    #     Pkt_grid = zeros(length(share)+1)

        #the first input is -Vmax, to assure that m will reject all offers. It is the waiting payoff
        Pkt_grid[1] = -Vmax

        #t+1 is just because we have T+1 periods in U, so U[1] is t==0 and U[T] is t==T-1
        #the reorganization payoff is U_{t} (\theta_{t+1})
        Pkt_grid[2:end] .= U[t, hkt] .* share


        #screening cutoffs
        cmt = Vector{Int64}(undef, length(Pkt_grid))

         #loop for calculating the cmt
        for (i, Pkt) in enumerate(Pkt_grid)
            cmt[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
        end


        payoff = zeros(length(Pkt_grid))


        #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
        probm = h_m(t,lmt, pmf, pmf2)

        for (i, pkt) in enumerate(Pkt_grid)
            if(cmt[i]==grid+2) 
                #means that pkt<0, so it will be always rejected
                #lmt+1 == lmt
                payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) 



            elseif(cmt[i] == grid+1) 
                #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

                #since sum(probm[lmt:end])==1, we didn't include it here
                payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end]))

            else
                #cases where cmt[i] is between 1 and grid

                if(cmt[i] < lmt)
                    #means that pkt<minimum continuation value of m, so it will be always rejected
                    #similar to pkt<0
                    #lmt+1 == lmt
                    payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end]))

                elseif(cmt[i]==lmt)

                    #accepted only if hmt+1==lmt
                    payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * probm[lmt]

                    #rejected otherwise
                    payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, lmt]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(lmt+1):end]))

                else
                    #cmt[i] > lmt
                    #accepted if hmt+1 <=cmt[i]
                    payoff[i] += dot(@view(U[t, hkt:end]) .- pkt, @view(pmf[hkt, hkt:end])) * sum(@view(probm[lmt:cmt[i]]))


                    #rejected otherwise
                    #lmt+1 = cmt[i]
                    payoff[i] += dot(@view(k_W[t+1, hkt:end, hkt, cmt[i]]), @view(pmf[hkt, hkt:end])) * sum(@view(probm[(cmt[i]+1):end]))

                end
            end
        end



        #payoffs matrix####

    #     payoff_reorg, index_reorg = findmax(payoff[2:end])
        payoff_reorg, index_reorg = findmax(payoff)
        payment = Pkt_grid[index_reorg]



        #calculating the optimal policy between liquidating, reorganizing or waiting ####
        payoff_liq = k_L(t)

        #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
        payoff_wait = payoff[1]

        payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))


        #condition to always show the payoff reorg
    #     if(policy==3.0)
    #         payment = Pkt_grid[index_reorg+1]
    #     else
    #         payment = -Vmax
    #     end

        #populates the array
        Pkt_array[t, hkt, lmt, :] .= payment, cmt[index_reorg], payoff_reorg, payoff_wait, payoff_max, policy

    end





    #optimal proposal
    function propose(t, hkt, lkt, lmt, Pkt_array)


        #it just searches for the corresponding Pkt
        payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]

        return payoff_max, policy
    end



    #"cdf" for the discretized distribution
    function cdf_discrete(x, lt, pmf)

        #returns the probability that θt takes a value less than or equal to x, given lt
        #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
        return sum(@view(pmf[lt, lt:x]))
    end



    #function to calculate the payment offer threshold ϕmt

    function threshold_m(t, lkt, lmt, Pmt_array)

        #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
        #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
        #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
        ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)


        #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
        @assert ts < 101

        return ts

    end



    function prob_liq_m(t, ts, lmt, pmf)

        #probability that m will propose liquidation next period

        #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
        if(t<=1)
            if(lmt >= ts)
                return 0.0
            else
                return 1.0
            end


        else#if t>0(index>1)

            if(lmt >= ts)
                return 0.0
            else
                return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
            end

        end


    end



    #expected payoff of answering a payment proposal

    function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf)


        #the minimum level of hmt necessary for m to offer a payment
        #it will be used as hmt
        ts = threshold_m(t, lkt, lmt, Pmt_array)

        #updated lower bound
        lmt = max(ts, lmt)


        #the possible values of payment given by the opponent
        Pmt_grid = Pmt_array[t, lmt:end, lkt, 1]


        #the screening cutoffs associated with each payment proposal
        #indexet by k because now it is k who is being screened
        ckt = Vector{Int64}(undef, length(Pmt_grid))
        ckt .= Pmt_array[t, lmt:end, lkt, 2]

        payoff = zero(0.0)
    #     payoff = Array{Float64}(undef, length(Pmt_grid))



        for (i, pmt) in enumerate(Pmt_grid)

            if(ckt[i]==grid+2)
                #means that the payment offer is negative (waiting offer)
                #the proposal will be rejected for sure, without lowerbound update
                payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]



            elseif(ckt[i]==grid+1)
                #means that there is no cutoff, K will accept the payment offer for all θkt+1
    #             payoff += pmt * sum(@view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]
                payoff += pmt * pmf[lmt, lmt+(i-1)]

            else

                if(ckt[i] < lkt)
                    #the proposal will be rejected for sure, without lowerbound update
                    payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]

                elseif(ckt[i]==lkt)
                    #will accept only if hkt+1==lkt
                    payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt+(i-1)]

                    #rejects otherwise
                    payoff += dot(@view(k_W[t+1, lkt+1:end, lkt, lmt+(i-1)]), @view(pmf[hkt, lkt+1:end])) * pmf[lmt, lmt+(i-1)]

                else
                #now ckt > lkt

                    if(ckt[i] < hkt)
                        #he rejects, but there is an update in the lowerbound lkt+1==ckt
                        payoff += dot(@view(k_W[t+1, hkt:end, ckt[i], lmt+(i-1)]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt+(i-1)]


                    else

                        #receives pmt if his skill level is below or equal to the cutoff
                        payoff += pmt * sum(@view(pmf[hkt, hkt:ckt[i]])) * pmf[lmt, lmt+(i-1)]


                        #receives continuation value k_W if his skill level greather than the cutoff
                        payoff += dot(@view(k_W[t+1, ckt[i]+1:end , ckt[i], lmt+(i-1)]), @view(pmf[hkt, ckt[i]+1:end])) * pmf[lmt, lmt+(i-1)]


                    end
                end
            end
        end

        return payoff
    end






    #expected payoff of answering a liquidation proposal

    function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)

        #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
        #U_{t} (\theta_{t+1}) because the skill level is of the next period
        #dot is matrix multiplication
        payoff_reorg = dot(@view(pmf[hkt, hkt:end]), (@view(U[t, hkt:end]) .- m_L(t)))

        payoff_liq = k_L(t)

        return max(payoff_liq, payoff_reorg)
    end




    # @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

    #expected payoff of being called to respond
    function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U)

        probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf)


        return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U)
    end



    #defining S functions based on the generic functions####

    #Pkt(t, hkt, lmt, k_W, m_W, k_L)
    function Pst(t, θst, ℓjt)
        return Pkt(t, θst, ℓjt, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U)
    end

    #propose(t, hkt, lkt, lmt, Pkt_array)
    function s_propose(t, θst, ℓst, ℓjt)
    #     r = propose(t, θst, ℓst, ℓjt, Pst_array)

    #     return r

        return propose(t, θst, ℓst, ℓjt, Pst_array)
    end

    #threshold_m(t, lkt, lmt, Pmt_array)
    function threshold_j(t, ℓst, ℓjt)
        return threshold_m(t, ℓst, ℓjt, Pjt_array)
    end


    #respond_reorg(t, hkt, lkt, lmt, Pmt_array, k_W)
    function s_respond_payment(t, θst, ℓst, ℓjt)
        return respond_payment(t, θst, ℓst, ℓjt, Pjt_array, s_W, pmf)
    end


    #respond_liq(t, hkt, lkt, lmt, k_L, m_L)
    function s_respond_liq(t, θst, ℓst, ℓjt)
        return respond_liq(t, θst, ℓst, ℓjt, s_L, j_L, pmf, U)
    end


    #respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L) 
    function s_respond(t, θst, ℓst, ℓjt)
        return respond(t, θst, ℓst, ℓjt, Pjt_array, s_W, s_L, j_L, pmf, U)
    end

    #J's functions based on the generic functions
    function Pjt(t, θjt, ℓst)
        return Pkt(t, θjt, ℓst, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U)
    end


    function j_propose(t, θjt, ℓjt, ℓst)
        return propose(t, θjt, ℓjt, ℓst, Pjt_array)
    end

    function threshold_s(t, ℓjt, ℓst)
        return threshold_m(t, ℓjt, ℓst, Pst_array)
    end

    function j_respond_payment(t, θjt, ℓjt, ℓst)
        return respond_payment(t, θjt, ℓjt, ℓst, Pst_array, j_W, pmf)
    end

    function j_respond_liq(t, θjt, ℓjt, ℓst)
        return respond_liq(t, θjt, ℓjt, ℓst, j_L, s_L, pmf, U)
    end    

    function j_respond(t, θjt, ℓjt, ℓst)
        return respond(t, θjt, ℓjt, ℓst, Pst_array, j_W, j_L, s_L, pmf, U)
    end
    
    
    

    function populate_periods!(tfinal, tbegin)

        @time begin


            for t in tfinal:-1:tbegin

                @time begin
                    #populating Pkt
                    for l in 1:grid
                        for h in 1:grid
                            Pst(t, h, l)
                            Pjt(t, h, l)
                        end
                    end

                    #populating Wkt
                    for lm in 1:grid
                        for lk in 1:grid
                            for hk in lk:grid #populates only when hk>=lk, to save time
                                s_W[t, hk, lk, lm] = λj * s_respond(t, hk, lk, lm) + (1-λj) * s_propose(t, hk, lk, lm)[1]
                                j_W[t, hk, lk, lm] = λj * j_propose(t, hk, lk, lm)[1] + (1-λj) * j_respond(t, hk, lk, lm)
                            end
                        end
                    end
                end

            end

        end

    end
    
    

    ### Populating the whole game

    populate_periods!(T,1)
    
    
    ### returns the arrays necessary to simulate the game
    return s_W, j_W, Pst_array, Pjt_array, Ds, Dj
    
end
